# Install SUMMA model configurations

This notebook contains a couple of functions to install SUMMA model configurations that you can then further explore yourself. These model configurations are:

* _SUMMA test cases_: These are used by the SUMMA development team to test SUMMA model code developments, but are also used in a number of the model tutorials. This setup includes a number of test cases, including the code to re-recreate all the plots in the second SUMMA paper [_[Clark et al., 2015](https://doi.org/10.1002/2015wr017200)_].

  To install the SUMMA test cases, execute `install_summa_test_cases()` in a code cell at the end of this notebook. The test cases will be installed in the directory: `/home/jovyan/summaTestCases_2.x_b074b0ffa`
  
* _SUMMA CUES site_: A SUMMA configuration for the CUES site, which has a lot of snow observations available [_[Bair et al., 2017](https://doi.org/10.3389/feart.2015.00058)_].

    To install the SUMMA setup for the CUES, execute `install_summa_cues_site()` in a code cell at the end of this notebook. The SUMMA configuration will be installed in the directory: `/home/jovyan/cues`

* _SUMMA Dana Meadows site_: A SUMMA configuration for the Dana Meadows site, which is a site menaged by the California Department of Water Resources [e.g. _[Cristea et al., 2013](https://doi.org/10.1002/hyp.9909); [Lundquist et al., 2018](https://doi.org/10.1016/j.rse.2018.03.001)_].

    To install the SUMMA setup for the Dana Meadows site, execute `install_summa_dana_meadows_site()` in a code cell at the end of this notebook. The SUMMA configuration will be installed in the directory: `/home/jovyan/dana-meadows`

* _SUMMA PLUMBER site_: These are SUMMA configurations for all 20 flux tower locations used by the PLUMBER study [_[Best et al., 2015](https://doi.org/10.1175/jhm-d-14-0158.1)_].

  To install the setup for a single PLUMBER site, execute `install_summa_plumber_site(site)` in a code cell at the end of this notebook, where `site` is one of the PLUMBER sites (see below for the available sites). The SUMMA configuration for each PLUMBER site will be installed in the directory: `/home/jovyan/PLUMBER_data/sites/<site>`, where `<site>` is the site you selected.

* _SUMMA Reynolds Creek site_: This is a SUMMA setup for the [Reynolds Creek Watershed](https://criticalzone.org/reynolds/) in central Idaho. The setup is for a single location and is best used for snow modeling.

* _SUMMA Umpqua site_: This is a SUMMA setup for two locations in the Umpqua watershed in coastal Oregon. The setup is best used for snow modeling and uses data collected by Pascal Storck to investigate snow interception by vegetation in maritime climates [_[Storck et al., 2002](https://doi.org/10.1029/2002WR001281)_].

    
Of course, after you type the code, you will need to run the cell as well. You can browse the directory structure for each of the model configurations in the built-in file manager to the left.

As time permits I will add some more documentation for each of the setups.

In [1]:
import subprocess
import tarfile
import glob
import os
import re

cues_source         = 'https://www.dropbox.com/s/sjkry53angvhdqr/cues.tar.gz?dl=1'
dana_meadows_source = 'https://www.dropbox.com/s/akox4bhjbhiqhg9/dana-meadows.tar.gz?dl=1'
plumber_sources     = {
        'Amplero':  'https://www.dropbox.com/s/gh30l9ebucpe53b/Amplero.tar.gz?dl=1',
        'Blodgett': 'https://www.dropbox.com/s/2zeg6c7qwjz7fkv/Blodgett.tar.gz?dl=1',
        'Bugac':    'https://www.dropbox.com/s/cywqvu57qadl6lj/Bugac.tar.gz?dl=1',
        'ElSaler':  'https://www.dropbox.com/s/qybwksk202ow8lh/ElSaler.tar.gz?dl=1',
        'ElSaler2': 'https://www.dropbox.com/s/a7dxwxwohgx2y3z/ElSaler2.tar.gz?dl=1',
        'Espirra':  'https://www.dropbox.com/s/zcg3rmwnfz6efrp/Espirra.tar.gz?dl=1',
        'FortPeck': 'https://www.dropbox.com/s/gb4tfi98zuxgjyc/FortPeck.tar.gz?dl=1',
        'Harvard':  'https://www.dropbox.com/s/p98l8gz55y1wsbd/Harvard.tar.gz?dl=1',
        'Hesse':    'https://www.dropbox.com/s/wzrsd1qdjl57o4t/Hesse.tar.gz?dl=1',
        'Howard':   'https://www.dropbox.com/s/dkjlvwsa1x7qpmd/Howard.tar.gz?dl=1',
        'Howlandm': 'https://www.dropbox.com/s/7iekl41365ptawa/Howlandm.tar.gz?dl=1',
        'Hyytiala': 'https://www.dropbox.com/s/bea3fbprfqfap1n/Hyytiala.tar.gz?dl=1',
        'Kruger':   'https://www.dropbox.com/s/lqcqrx9n3mqpqe8/Kruger.tar.gz?dl=1',
        'Loobos':   'https://www.dropbox.com/s/r2knag2ouhxxwrz/Loobos.tar.gz?dl=1',
        'Merbleue': 'https://www.dropbox.com/s/z1met5akkzhc86y/Merbleue.tar.gz?dl=1',
        'Mopane':   'https://www.dropbox.com/s/syddkx7lum8ymzg/Mopane.tar.gz?dl=1',
        'Palang':   'https://www.dropbox.com/s/es8d8ggq8z9blzf/Palang.tar.gz?dl=1',
        'Sylvania': 'https://www.dropbox.com/s/znclkmzaciyrgvy/Sylvania.tar.gz?dl=1',
        'Tumba':    'https://www.dropbox.com/s/m3gjtbcgrw44hov/Tumba.tar.gz?dl=1',
        'UniMich':  'https://www.dropbox.com/s/p1rwk685wsdvvnh/UniMich.tar.gz?dl=1',
    }
reynolds_source     = 'https://www.dropbox.com/s/ra3k213s2w17693/reynolds.tar.gz?dl=1'
summa_test_source   = 'https://www.dropbox.com/s/dooavp5l8wtiv9d/summaTestCases_2.1.tar.gz?dl=1'
umpqua_source       = 'https://www.dropbox.com/s/sl3hjy4jywcu4pf/umpqua.tar.gz?dl=1'

def get_untar(source, destination, remove=True):

    # download the tar file
    tarfilename = '{}/download.tar.gz'.format(destination)
    command = 'wget --max-redirect=20 -O {} {}'.format(tarfilename, source)
    subprocess.run(command.split())
    
    # unpack the tar file
    tar = tarfile.open(tarfilename, 'r:gz')
    tar.extractall(destination)
    
    if remove:
        # remove the tar file
        os.unlink(tarfilename)
    

def install_summa_test_cases(destination='/home/jovyan'):

    # get tar file and unpack
    get_untar(summa_test_source, destination)

    # install the test cases
    pwd = os.getcwd()
    os.chdir('{}/summaTestCases_2.x_b074b0ffa'.format(destination))
    subprocess.run('./installTestCases_local.sh')
    os.chdir(pwd)
    
    # Print message
    print('Test cases installed in {}'.format(destination))

def install_summa_cues_site(destination='/home/jovyan'):
    
    # get tar file and unpack
    get_untar(cues_source, destination)

    # install the test cases
    pwd = os.getcwd()
    os.chdir('{}/cues/summa_setup'.format(destination))
    subprocess.run('./install_local_setup.sh')
    os.chdir(pwd)
    
    # Print message
    print('cues installed in {}'.format(destination))

def install_summa_dana_meadows_site(destination='/home/jovyan'):
    
    # get tar file and unpack
    get_untar(dana_meadows_source, destination)

    # install the test cases
    pwd = os.getcwd()
    os.chdir('{}/dana-meadows/summa_setup'.format(destination))
    subprocess.run('./install_local_setup.sh')
    os.chdir(pwd)
    
    # Print message
    print('dana-meadows installed in {}'.format(destination))
 
def install_summa_plumber_site(site, destination='/home/jovyan'):

    # get tar file and unpack
    get_untar(plumber_sources[site], destination)
    
    # fix the decisions file(s)
    ifiles = glob.glob('{}/**/{}/**/*zDecisions*.txt'.format(destination, site), recursive=True)
    for ifile in ifiles:
        with open(ifile, "r") as f:
            content = f.readlines()
        with open(ifile, "w") as f:
            for line in content:
                f.write(re.sub('1998', '1988', line))
                
    # fix the path in the file manager(s)
    ifiles = glob.glob('{}/**/{}/**/*zFileManager*.txt'.format(destination, site), recursive=True)
    for ifile in ifiles:
        with open(ifile, "r") as f:
            content = f.readlines()
        with open(ifile, "w") as f:
            for line in content:
                f.write(re.sub('/PLUMBER_data', '{}/PLUMBER_data'.format(destination), line))
    
    # Print message
    print('{} installed in {}'.format(site, destination))

def install_summa_reynolds_site(destination='/home/jovyan'):

    # get tar file and unpack
    get_untar(reynolds_source, destination)

    # fix the path in the file manager(s)
    ifiles = glob.glob('{}/**/reynolds/**/*fileManager*.txt'.format(destination), recursive=True)
    for ifile in ifiles:
        with open(ifile, "r") as f:
            content = f.readlines()
        with open(ifile, "w") as f:
            for line in content:
                f.write(re.sub('<BASEPATH>', '{}'.format(destination), line))

    # Print message
    print('reynolds installed in {}'.format(destination))

def install_summa_umpqua_site(destination='/home/jovyan'):

    # get tar file and unpack
    get_untar(umpqua_source, destination)

    # fix the path in the file manager(s)
    ifiles = glob.glob('{}/**/umpqua/**/*fileManager*.txt'.format(destination), recursive=True)
    for ifile in ifiles:
        with open(ifile, "r") as f:
            content = f.readlines()
        with open(ifile, "w") as f:
            for line in content:
                f.write(re.sub('<BASEPATH>', '{}'.format(destination), line))
                
    # Print message
    print('umpqua installed in {}'.format(destination))


In [ ]:
# To install the SUMMA test cases execute the following command:
install_summa_test_cases()

In [ ]:
# To install the SUMMA configurations for the CUES site execute the following command:
install_summa_cues_site()

In [ ]:
# To install the SUMMA configurations for the Dana Meadows site execute the following command:
install_summa_dana_meadows_site()

In [ ]:
# To install the SUMMA configurations for a PLUMBER site (e.g. Harvard) site execute the following command:
install_summa_plumber_site('Harvard')

In [ ]:
# To install the SUMMA configurations for the Reynolds site execute the following command:
install_summa_reynolds_site()

In [ ]:
# To install the SUMMA configurations for the Umpqua site execute the following command:
install_summa_umpqua_site()